# 나이브베이즈 분류를 이용한 감성 분석 - 한국어

## 1. 한국어 분석을 위한 konlpy 패키지 import

In [5]:
from konlpy.tag import Okt
from nltk.tokenize import word_tokenize
import nltk

pos_tagger = Okt()

## data 정의
### train data
- Naive Bayes 분류기는 지도학습 알고리즘이므로 정답(label)을 알려줘야함

In [4]:
train = [
    ("메리가 좋아", "pos"),
    ("고양이도 좋아", "pos"),
    ("난 수업이 지루해", "neg"),
    ("메리는 이쁜 고양이야", "pos"),
    ("난 마치고 메리랑 놀거야", "pos"),
]

### 말뭉치 생성

In [6]:
all_words = set(
    word.lower() for sentence in train for word in word_tokenize(sentence[0])
)

all_words

{'고양이도',
 '고양이야',
 '난',
 '놀거야',
 '마치고',
 '메리가',
 '메리는',
 '메리랑',
 '수업이',
 '이쁜',
 '좋아',
 '지루해'}

### 형태소 기준 tokenize

In [7]:
def tokenize(doc) :
    return ["/".join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

In [8]:
train_docs = [(tokenize(row[0]), row[1]) for row in train]

train_docs

[(['메리/Noun', '가/Josa', '좋다/Adjective'], 'pos'),
 (['고양이/Noun', '도/Josa', '좋다/Adjective'], 'pos'),
 (['난/Noun', '수업/Noun', '이/Josa', '지루하다/Adjective'], 'neg'),
 (['메리/Noun', '는/Josa', '이쁘다/Adjective', '고양이/Noun', '야/Josa'], 'pos'),
 (['난/Noun', '마치/Noun', '고/Josa', '메리/Noun', '랑/Josa', '놀다/Verb'], 'pos')]

### tokenize된 형태소들로 말뭉치 생성

In [10]:
tokens = [t for d in train_docs for t in d[0]]

tokens

['메리/Noun',
 '가/Josa',
 '좋다/Adjective',
 '고양이/Noun',
 '도/Josa',
 '좋다/Adjective',
 '난/Noun',
 '수업/Noun',
 '이/Josa',
 '지루하다/Adjective',
 '메리/Noun',
 '는/Josa',
 '이쁘다/Adjective',
 '고양이/Noun',
 '야/Josa',
 '난/Noun',
 '마치/Noun',
 '고/Josa',
 '메리/Noun',
 '랑/Josa',
 '놀다/Verb']

### 말뭉치 대비 train 데이터에 단어가 있고 없음을 표기

- train 데이터의 각 문장에 말뭉치에 있는 단어가 있으면 True, 없으면 False로 구분

In [11]:
def term_exists(doc) :
    return {word : (word in set(doc)) for word in tokens}

In [12]:
train_xy = [(term_exists(d), c) for d, c in train_docs]

train_xy

[({'메리/Noun': True,
   '가/Josa': True,
   '좋다/Adjective': True,
   '고양이/Noun': False,
   '도/Josa': False,
   '난/Noun': False,
   '수업/Noun': False,
   '이/Josa': False,
   '지루하다/Adjective': False,
   '는/Josa': False,
   '이쁘다/Adjective': False,
   '야/Josa': False,
   '마치/Noun': False,
   '고/Josa': False,
   '랑/Josa': False,
   '놀다/Verb': False},
  'pos'),
 ({'메리/Noun': False,
   '가/Josa': False,
   '좋다/Adjective': True,
   '고양이/Noun': True,
   '도/Josa': True,
   '난/Noun': False,
   '수업/Noun': False,
   '이/Josa': False,
   '지루하다/Adjective': False,
   '는/Josa': False,
   '이쁘다/Adjective': False,
   '야/Josa': False,
   '마치/Noun': False,
   '고/Josa': False,
   '랑/Josa': False,
   '놀다/Verb': False},
  'pos'),
 ({'메리/Noun': False,
   '가/Josa': False,
   '좋다/Adjective': False,
   '고양이/Noun': False,
   '도/Josa': False,
   '난/Noun': True,
   '수업/Noun': True,
   '이/Josa': True,
   '지루하다/Adjective': True,
   '는/Josa': False,
   '이쁘다/Adjective': False,
   '야/Josa': False,
   '마치/Noun': False,
   '고/Jo

## 3. NaiveBayesClassifier

### Naive Bayes 분류기 훈련

In [13]:
classifier = nltk.NaiveBayesClassifier.train(train_xy)

classifier.show_most_informative_features()

Most Informative Features
                  난/Noun = True              neg : pos    =      2.5 : 1.0
                 메리/Noun = False             neg : pos    =      2.5 : 1.0
                고양이/Noun = False             neg : pos    =      1.5 : 1.0
            좋다/Adjective = False             neg : pos    =      1.5 : 1.0
                  가/Josa = False             neg : pos    =      1.1 : 1.0
                  고/Josa = False             neg : pos    =      1.1 : 1.0
                 놀다/Verb = False             neg : pos    =      1.1 : 1.0
                  는/Josa = False             neg : pos    =      1.1 : 1.0
                  도/Josa = False             neg : pos    =      1.1 : 1.0
                  랑/Josa = False             neg : pos    =      1.1 : 1.0


### test

In [14]:
test_sentence = "난 수업이 마치면 메리랑 놀거야"

test_sent_features = {
    word.lower() : (word in word_tokenize(test_sentence.lower())) for word in all_words
}

test_sent_features

{'고양이야': False,
 '메리랑': True,
 '놀거야': True,
 '메리는': False,
 '이쁜': False,
 '메리가': False,
 '수업이': True,
 '마치고': False,
 '지루해': False,
 '좋아': False,
 '난': True,
 '고양이도': False}

In [15]:
classifier.classify(test_sent_features)

'pos'